In [1]:
from openai import OpenAI
# 设置 API 密钥和组织
api_key = "sk-WO49kazJ9a1PDGbE2lPRhJ9vDG8hak5_mbD60YhIB8T3BlbkFJHbhSLNYAPwzhLHjS72qabHWqGwtBmAvgRpRwN_iYMA"
client = OpenAI(api_key=api_key)

# 设置组织信息
client.organization = "org-34qJHjOz4UGZYVqv2zlohVMe"

In [4]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor

# 定义文件路径
root_folder = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/raw_data_tales'
all_files = []

# 遍历目录，收集所有文件路径
for dirpath, _, filenames in os.walk(root_folder):
    for filename in filenames:
        relative_path = os.path.join(*dirpath.split(os.sep)[-2:], filename)
        all_files.append((os.path.join(dirpath, filename), relative_path))

# 随机选取100个文件
sample_size = min(len(all_files), 100)
selected_files = [all_files[i] for i in np.random.choice(len(all_files), sample_size, replace=False)]

# 读取文件内容并存储
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()
file_contents = {relative_path: read_file(file_path) for file_path, relative_path in selected_files}

# 初始化文件评分字典
scores = defaultdict(int)
file_pairs = [(selected_files[i][1], selected_files[j][1]) for i in range(len(selected_files)) for j in range(i + 1, len(selected_files))]

# 文件内容比较函数
def compare_files(file_pair):
    file1, file2 = file_pair
    file1_content = file_contents[file1]
    file2_content = file_contents[file2]
    prompt = f"你是一位精通中国民间故事研究的研究员，需要研究\"结拜\"主题的故事--具体而言，这个故事应包含通过特殊仪式结为兄弟姐妹的‘结拜’情节，体现超越血缘的亲密关系，基于共同信念与目标，人与人之间通过誓言与约定建立起深厚的情谊和相互扶持的纽带，反映对义气、兄弟情深与忠诚的高度推崇。根据以下两篇中国民间故事文本，请判断哪一段与\"结拜\"主题的相关程度更高。\n\n文本1：\n{file1_content}\n\n文本2：\n{file2_content}\n\n注意，请直接回答“文本1”或“文本2”，不要输出其他文字。此外，请仅依据对主题的具体解释作出判断，不使用任何外部信息、不允许推断、不允许牵强附会或过度解读。"
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=3,
            temperature=0.1,
            top_p=0.9
        )
        answer = response.choices[0].message.content.strip()
        return file_pair, answer
    except Exception as e:
        print(f"Error processing {file_pair}: {e}")
        return file_pair, "Error"

# 多线程处理文件对
with ThreadPoolExecutor(max_workers=8) as executor:
    results = list(tqdm(executor.map(compare_files, file_pairs), total=len(file_pairs), desc="比较文件"))

# 更新评分
for (file1, file2), result in results:
    if result == "文本1":
        scores[file1] += 1
    elif result == "文本2":
        scores[file2] += 1

# 保存结果到 CSV
sample_results = [{"文件路径": file, "内容": file_contents[file], "评分": scores[file]} for file in file_contents]
sample_df = pd.DataFrame(sample_results)
sample_df.to_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241125gpt_sample_scores_jiebai.csv', index=False, encoding='utf-8-sig')
print("抽样文件的评分结果已保存到 sample_scores.csv")

比较文件: 100%|██████████| 4950/4950 [06:54<00:00, 11.94it/s] 

抽样文件的评分结果已保存到 sample_scores.csv


In [2]:
# 加载 BERT 模型
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('DMetaSoul/sbert-chinese-general-v2')

/Users/Anaconda/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-11-25 03:33:30.448222: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# 获取得分最高的文件内容和路径
sample_df = pd.read_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241125gpt_sample_scores_jiebai.csv')
highest_score_row = sample_df.loc[sample_df['评分'].idxmax()]
highest_score_content = highest_score_row['内容']

# 文件相似度比较函数
def compute_similarity(file_path, file_content, highest_score_content):
    embeddings = model.encode([file_content, highest_score_content])
    A, B = embeddings[0], embeddings[1]
    similarity = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
    return {'文件名': file_path, '相似度': similarity}

# 遍历子文件夹中的所有.txt文件，获取文件路径列表
folder_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/raw_data_tales'
file_paths = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.txt'):
            file_paths.append(os.path.join(root, file))

# 并行处理文件
results = []
with ThreadPoolExecutor(max_workers=60) as executor:  # 调整 max_workers 根据系统性能
    future_to_file = {
        executor.submit(
            compute_similarity,
            file_path,
            open(file_path, 'r', encoding='utf-8').read(),
            highest_score_content
        ): file_path
        for file_path in file_paths
    }

    for future in tqdm(as_completed(future_to_file), total=len(future_to_file), desc="Processing files"):
        try:
            result = future.result()
            results.append(result)
        except Exception as e:
            print(f"Error processing file {future_to_file[future]}: {e}")

# 将结果保存到 CSV 文件中
output_csv_file = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241125similarity_results_jiebai_real_story.csv'
results_df = pd.DataFrame(results)
results_df.to_csv(output_csv_file, index=False, encoding='utf-8')

print("相似度计算完成，结果已保存为 CSV 文件。")

Processing files: 100%|██████████| 17324/17324 [10:02<00:00, 28.76it/s]


相似度计算完成，结果已保存为 CSV 文件。


In [10]:
results_df.iloc[0]['文件名']

'01170__414-女人的围腰布.txt'

In [9]:
results_df['文件名'] = results_df['文件名'].apply(lambda x: os.path.basename(x))
